# Importing libraries

In [1]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

%matplotlib inline

# Loading data

In [2]:
train = pd.read_csv('../input/urban-air-pollution-challenge-by-zindiweekendz/Train (9).csv')
test = pd.read_csv('../input/urban-air-pollution-challenge-by-zindiweekendz/Test (4).csv')

# Feature engineering

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30557 entries, 0 to 30556
Data columns (total 82 columns):
Place_ID X Date                                        30557 non-null object
Date                                                   30557 non-null object
Place_ID                                               30557 non-null object
target                                                 30557 non-null float64
target_min                                             30557 non-null float64
target_max                                             30557 non-null float64
target_variance                                        30557 non-null float64
target_count                                           30557 non-null int64
precipitable_water_entire_atmosphere                   30557 non-null float64
relative_humidity_2m_above_ground                      30557 non-null float64
specific_humidity_2m_above_ground                      30557 non-null float64
temperature_2m_above_ground           

In [4]:
train.describe()

,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
count,30557.000000,30557.000000,30557.000000,3.055700e+04,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,30557.000000,...,23320.000000,23320.000000,23320.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000,5792.000000
mean,61.148045,29.025866,117.992234,7.983756e+03,125.831135,15.302326,70.552747,0.006004,9.321342,0.416886,...,35.590916,-123.697777,46.533951,923.231949,1711.793613,0.016227,1.254703,13.849040,-69.098594,23.100630
std,46.861309,33.119775,100.417713,4.863090e+04,146.581856,10.688573,18.807884,0.003787,9.343226,2.707990,...,18.955228,71.916036,14.594267,929.633988,1741.299304,0.027016,55.101250,18.004375,84.702355,24.786350
min,1.000000,1.000000,1.000000,0.000000e+00,2.000000,0.420044,5.128572,0.000139,-34.647879,-15.559646,...,0.000000,-179.880630,0.000000,0.000000,0.000000,0.000000,-105.367363,0.000000,-179.947422,0.000000
25%,25.000000,5.000000,60.000000,1.064920e+03,44.000000,7.666667,58.600002,0.003403,3.123071,-1.097864,...,19.451524,-165.882624,36.693094,0.000000,0.000000,0.000000,0.000000,0.000000,-161.726937,0.000000
50%,50.000000,15.000000,91.000000,2.395350e+03,72.000000,12.200000,74.099998,0.004912,8.478424,0.222092,...,37.918838,-156.637162,47.445010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,80.000000,44.000000,155.000000,5.882550e+03,150.000000,19.900000,85.450001,0.007562,16.201563,1.772925,...,52.270055,-118.453598,57.438181,1861.674119,3393.541633,0.023829,62.245728,27.412303,0.000000,47.090635
max,815.000000,438.000000,999.000000,1.841490e+06,1552.000000,72.599998,100.000000,0.021615,37.437921,17.955124,...,66.111289,179.776125,79.631711,2112.522949,6478.550544,0.210483,77.355232,59.972710,179.813344,69.992363


In [5]:
test.describe()

,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
count,16136.000000,16136.000000,16136.000000,16136.000000,16136.000000,16136.000000,14809.000000,14809.000000,14809.000000,14809.000000,...,11963.000000,11963.000000,11963.000000,3115.000000,3115.000000,3115.000000,3115.000000,3115.000000,3115.000000,3115.000000
mean,14.202076,71.136312,0.005513,8.651834,0.660754,0.113826,0.000098,0.000166,-1.089137,0.293308,...,34.894186,-122.992436,49.048986,945.230960,1766.062448,0.018122,0.587431,14.053838,-71.287981,24.113993
std,9.377060,18.357542,0.003261,8.033144,2.854376,2.914895,0.000084,0.000101,0.616871,0.306695,...,18.808611,76.258227,13.809821,930.249966,1755.875373,0.029822,55.802139,18.085361,87.392331,25.099069
min,0.900000,6.700000,0.000279,-26.628958,-15.837668,-14.043520,-0.000031,0.000000,-3.495448,0.000000,...,0.000000,-179.953644,0.000000,0.000000,0.000000,0.000000,-105.016572,0.000000,-179.924887,0.000000
25%,7.655303,60.000000,0.003401,3.201305,-1.040559,-1.590374,0.000059,0.000127,-1.525122,0.029981,...,18.517603,-166.951551,40.406972,0.000000,0.000000,0.000000,0.000000,0.000000,-163.013042,0.000000
50%,11.600000,74.800003,0.004610,7.482545,0.402638,0.043358,0.000079,0.000154,-1.172175,0.172885,...,36.782582,-158.740565,50.105448,1771.898988,2943.639456,0.003386,0.000000,1.538256,0.000000,21.074167
75%,18.100000,85.500000,0.006598,13.834132,2.167810,1.708833,0.000115,0.000191,-0.721279,0.500543,...,51.485177,-124.569494,59.020340,1863.157119,3424.017252,0.025774,62.156588,27.354263,0.000000,48.395878
max,63.100002,100.000000,0.020180,34.851022,19.889325,16.682952,0.001233,0.001343,5.196880,1.000000,...,66.216644,179.884901,79.647072,1992.809481,6196.581543,0.215117,75.897305,59.984665,179.955138,69.925462


In [6]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [7]:
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['week'] = train['Date'].dt.week
train['day'] = train['Date'].dt.day

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['week'] = test['Date'].dt.week
test['day'] = test['Date'].dt.day

In [8]:
train['total_days_month'] = train['month'].apply(lambda x: 31 if x==1 else (28+31 if x==2 else (28+31+31 if x==1 else 28+30+31+31))) 
train['total_days'] = train['total_days_month'] + train['day']

test['total_days_month'] = test['month'].apply(lambda x: 31 if x==1 else (28+31 if x==2 else (28+31+31 if x==1 else 28+30+31+31))) 
test['total_days'] = test['total_days_month'] + test['day']

In [9]:
train.drop('Date', axis=1, inplace=True)
test.drop('Date', axis=1, inplace=True)

In [10]:
corrmat = train.corr()
corrmat[corrmat['target']>0.2].target.index

Index(['target', 'target_min', 'target_max',
       'L3_NO2_NO2_column_number_density',
       'L3_NO2_NO2_slant_column_number_density',
       'L3_NO2_tropospheric_NO2_column_number_density',
       'L3_CO_CO_column_number_density',
       'L3_HCHO_HCHO_slant_column_number_density',
       'L3_HCHO_tropospheric_HCHO_column_number_density'],
      dtype='object')

In [11]:
def radian_conv(degree):
    """
    Return radian.
    """
    return  np.radians(degree) 
train['L3_NO2_sensor_azimuth_angle'] = radian_conv(train['L3_NO2_sensor_azimuth_angle'])
train['L3_NO2_sensor_zenith_angle'] = radian_conv(train['L3_NO2_sensor_zenith_angle'])
train['L3_NO2_solar_azimuth_angle'] = radian_conv(train['L3_NO2_solar_azimuth_angle'])
train['L3_NO2_solar_zenith_angle'] = radian_conv(train['L3_NO2_solar_zenith_angle'])
train['L3_O3_sensor_azimuth_angle'] = radian_conv(train['L3_O3_sensor_azimuth_angle'])
train['L3_O3_sensor_zenith_angle'] = radian_conv(train['L3_O3_sensor_zenith_angle'])
train['L3_O3_solar_azimuth_angle'] = radian_conv(train['L3_O3_solar_azimuth_angle'])
train['L3_O3_solar_zenith_angle'] = radian_conv(train['L3_O3_solar_zenith_angle'])
train['L3_CO_sensor_azimuth_angle'] = radian_conv(train['L3_CO_sensor_azimuth_angle'])
train['L3_CO_sensor_zenith_angle'] = radian_conv(train['L3_CO_sensor_zenith_angle'])
train['L3_CO_solar_azimuth_angle'] = radian_conv(train['L3_CO_solar_azimuth_angle'])
train['L3_CO_solar_zenith_angle'] = radian_conv(train['L3_CO_solar_zenith_angle'])

test['L3_NO2_sensor_azimuth_angle'] = radian_conv(test['L3_NO2_sensor_azimuth_angle'])
test['L3_NO2_sensor_zenith_angle'] = radian_conv(test['L3_NO2_sensor_zenith_angle'])
test['L3_NO2_solar_azimuth_angle'] = radian_conv(test['L3_NO2_solar_azimuth_angle'])
test['L3_NO2_solar_zenith_angle'] = radian_conv(test['L3_NO2_solar_zenith_angle'])
test['L3_O3_sensor_azimuth_angle'] = radian_conv(test['L3_O3_sensor_azimuth_angle'])
test['L3_O3_sensor_zenith_angle'] = radian_conv(test['L3_O3_sensor_zenith_angle'])
test['L3_O3_solar_azimuth_angle'] = radian_conv(test['L3_O3_solar_azimuth_angle'])
test['L3_O3_solar_zenith_angle'] = radian_conv(test['L3_O3_solar_zenith_angle'])
test['L3_CO_sensor_azimuth_angle'] = radian_conv(test['L3_CO_sensor_azimuth_angle'])
test['L3_CO_sensor_zenith_angle'] = radian_conv(test['L3_CO_sensor_zenith_angle'])
test['L3_CO_solar_azimuth_angle'] = radian_conv(test['L3_CO_solar_azimuth_angle'])
test['L3_CO_solar_zenith_angle'] = radian_conv(test['L3_CO_solar_zenith_angle'])

In [12]:
data=pd.concat([train,test],axis=0)
data

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,...,target_max,target_min,target_variance,temperature_2m_above_ground,total_days,total_days_month,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,week,year
0,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,1793.793579,3227.855469,0.010579,74.481049,...,53.0,23.0,769.50,18.516840,33,31,1.996377,-1.227395,1,2020
1,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,...,63.0,25.0,1319.85,22.546533,34,31,3.330430,-1.188108,1,2020
2,-1.001241,841410.713456,-105.201338,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,...,56.0,8.0,1181.96,27.031030,35,31,5.065727,3.500559,1,2020
3,-0.777023,841103.242368,-104.334056,29.180977,-73.896572,30.545393,NaN,NaN,NaN,NaN,...,55.0,10.0,1113.67,23.971857,36,31,3.004001,1.099468,1,2020
4,0.366324,840763.055499,58.850179,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,...,52.0,9.0,1164.82,16.816309,37,31,2.621787,2.670559,2,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16131,-1.011116,833312.353691,63.241090,51.611371,-176.521720,38.857262,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.136346,151,120,0.073416,0.923135,14,2020
16132,-1.363941,833125.012462,-17.869847,55.065521,15.079379,40.886310,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.394495,121,120,-1.217649,1.270308,14,2020
16133,-1.776128,832573.386814,-99.647714,27.132738,-152.254156,41.190326,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.750879,122,120,-0.410603,0.754363,14,2020
16134,-2.971064,832687.223024,38.739218,0.564620,-158.883352,39.431193,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,8.322565,123,120,1.659441,-2.471616,14,2020


In [13]:
label_encoder =LabelEncoder() 
data['Place_ID']= label_encoder.fit_transform(data['Place_ID']) 

In [14]:
a=[i for i in range(1,46694)]
data['num'] = a

In [15]:
def lag_feature(df, lags, col):
    tmp = df[['Place_ID','num',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['Place_ID','num',col+'_lag_'+str(i)]
        shifted['num'] += i
        df = pd.merge(df, shifted, on=['Place_ID','num'], how='left')
    return df

In [16]:
data = lag_feature(data, [1,2,3,4,5,6], 'precipitable_water_entire_atmosphere')
data = lag_feature(data, [1,2,3,4,5,6], 'temperature_2m_above_ground')
data = lag_feature(data, [1,2,3,30], 'L3_NO2_NO2_column_number_density')
data = lag_feature(data, [1,2,3,4,5,30], 'L3_O3_O3_column_number_density')
data = lag_feature(data, [1,2,3,4,5,6,30], 'L3_HCHO_tropospheric_HCHO_column_number_density')
data = lag_feature(data, [1,2,3,4,5,30], 'L3_CO_CO_column_number_density')
data = lag_feature(data, [1,2,3,4,5,30], 'L3_CLOUD_surface_albedo')
data = lag_feature(data, [1,2,3,4,5,30], 'L3_CLOUD_cloud_optical_depth')
data = lag_feature(data, [1,2,3], 'relative_humidity_2m_above_ground')
data = lag_feature(data, [10,20,30], 'L3_AER_AI_sensor_altitude')
data = lag_feature(data, [1,2,3,4,30], 'L3_NO2_tropopause_pressure')

In [17]:
train=data.iloc[:30557 ,:]
test=data.iloc[30557:,]

# Modeling

In [18]:
columns=['precipitable_water_entire_atmosphere', 'relative_humidity_2m_above_ground','precipitable_water_entire_atmosphere_lag_1',
      'precipitable_water_entire_atmosphere_lag_2','precipitable_water_entire_atmosphere_lag_3','precipitable_water_entire_atmosphere_lag_4',
 'precipitable_water_entire_atmosphere_lag_5',
       'precipitable_water_entire_atmosphere_lag_6',
        'temperature_2m_above_ground_lag_1',
         'temperature_2m_above_ground_lag_2',
         'temperature_2m_above_ground_lag_3',
         'temperature_2m_above_ground_lag_4',
         'temperature_2m_above_ground_lag_5',
         'temperature_2m_above_ground_lag_6',
         'L3_NO2_NO2_column_number_density_lag_1',
         'L3_NO2_NO2_column_number_density_lag_2',
         'L3_NO2_NO2_column_number_density_lag_3',
         'L3_NO2_NO2_column_number_density_lag_30',
         
         'L3_O3_O3_column_number_density_lag_1',
       'L3_O3_O3_column_number_density_lag_2',
       'L3_O3_O3_column_number_density_lag_3',
         'L3_O3_O3_column_number_density_lag_4',
         'L3_O3_O3_column_number_density_lag_5',
         'L3_O3_O3_column_number_density_lag_30',
         
         'L3_HCHO_tropospheric_HCHO_column_number_density_lag_1',
       'L3_HCHO_tropospheric_HCHO_column_number_density_lag_2',
       'L3_HCHO_tropospheric_HCHO_column_number_density_lag_3',
         'L3_HCHO_tropospheric_HCHO_column_number_density_lag_4',
       'L3_HCHO_tropospheric_HCHO_column_number_density_lag_5',
         'L3_HCHO_tropospheric_HCHO_column_number_density_lag_6',
         'L3_HCHO_tropospheric_HCHO_column_number_density_lag_30',
         
         'L3_CO_CO_column_number_density_lag_1',
       'L3_CO_CO_column_number_density_lag_2',
       'L3_CO_CO_column_number_density_lag_3',
       'L3_CO_CO_column_number_density_lag_4',
       'L3_CO_CO_column_number_density_lag_5',
         'L3_CO_CO_column_number_density_lag_30',
         
         'L3_CLOUD_surface_albedo_lag_1',
       'L3_CLOUD_surface_albedo_lag_2', 'L3_CLOUD_surface_albedo_lag_3',
       'L3_CLOUD_surface_albedo_lag_4', 'L3_CLOUD_surface_albedo_lag_5',
         'L3_CLOUD_surface_albedo_lag_30',
         
         
         'L3_CLOUD_cloud_optical_depth_lag_1',
       'L3_CLOUD_cloud_optical_depth_lag_2',
       'L3_CLOUD_cloud_optical_depth_lag_3',
         'L3_CLOUD_cloud_optical_depth_lag_4',
         'L3_CLOUD_cloud_optical_depth_lag_5',
       'L3_CLOUD_cloud_optical_depth_lag_30',
         
       
          'relative_humidity_2m_above_ground_lag_1',
       'relative_humidity_2m_above_ground_lag_2',
       'relative_humidity_2m_above_ground_lag_3',
       
         'L3_AER_AI_sensor_altitude_lag_10', 
         'L3_AER_AI_sensor_altitude_lag_20',
       'L3_AER_AI_sensor_altitude_lag_30',
         
         'L3_NO2_tropopause_pressure_lag_1',
       'L3_NO2_tropopause_pressure_lag_2', 'L3_NO2_tropopause_pressure_lag_3',
         'L3_NO2_tropopause_pressure_lag_4','L3_NO2_tropopause_pressure_lag_30',
         

      

        

       'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground',
       'L3_NO2_NO2_column_number_density',
       'L3_NO2_NO2_slant_column_number_density',
       'L3_NO2_absorbing_aerosol_index', 'L3_NO2_cloud_fraction',
       'L3_NO2_sensor_altitude', 'L3_NO2_sensor_azimuth_angle',
       'L3_NO2_sensor_zenith_angle', 'L3_NO2_solar_azimuth_angle',
       'L3_NO2_solar_zenith_angle',
       'L3_NO2_stratospheric_NO2_column_number_density',
       'L3_NO2_tropopause_pressure',
       'L3_NO2_tropospheric_NO2_column_number_density',
       'L3_O3_O3_column_number_density', 'L3_O3_O3_effective_temperature',
       'L3_O3_cloud_fraction', 'L3_O3_sensor_azimuth_angle',
       'L3_O3_sensor_zenith_angle', 'L3_O3_solar_azimuth_angle',
       'L3_O3_solar_zenith_angle', 'L3_CO_CO_column_number_density',
       'L3_CO_H2O_column_number_density', 'L3_CO_cloud_height',
       'L3_CO_sensor_altitude', 'L3_CO_sensor_azimuth_angle',
       'L3_CO_sensor_zenith_angle', 'L3_CO_solar_azimuth_angle',
       'L3_CO_solar_zenith_angle', 'L3_HCHO_HCHO_slant_column_number_density',
       'L3_HCHO_cloud_fraction', 'L3_HCHO_sensor_azimuth_angle',
       'L3_HCHO_sensor_zenith_angle', 'L3_HCHO_solar_azimuth_angle',
       'L3_HCHO_solar_zenith_angle',
       'L3_HCHO_tropospheric_HCHO_column_number_density',
       'L3_HCHO_tropospheric_HCHO_column_number_density_amf',
       'L3_CLOUD_cloud_base_height', 'L3_CLOUD_cloud_base_pressure',
       'L3_CLOUD_cloud_fraction', 'L3_CLOUD_cloud_optical_depth',
       'L3_CLOUD_cloud_top_height', 'L3_CLOUD_cloud_top_pressure',
       'L3_CLOUD_sensor_azimuth_angle', 'L3_CLOUD_sensor_zenith_angle',
       'L3_CLOUD_solar_azimuth_angle', 'L3_CLOUD_solar_zenith_angle',
       'L3_CLOUD_surface_albedo', 'L3_AER_AI_absorbing_aerosol_index',
       'L3_AER_AI_sensor_altitude', 'L3_AER_AI_sensor_azimuth_angle',
       'L3_AER_AI_sensor_zenith_angle', 'L3_AER_AI_solar_azimuth_angle',
       'L3_AER_AI_solar_zenith_angle', 'L3_SO2_SO2_column_number_density',
       'L3_SO2_SO2_column_number_density_amf',
       'L3_SO2_SO2_slant_column_number_density',
       'L3_SO2_absorbing_aerosol_index', 'L3_SO2_cloud_fraction',
       'L3_SO2_sensor_azimuth_angle', 'L3_SO2_sensor_zenith_angle',
       'L3_SO2_solar_azimuth_angle', 'L3_SO2_solar_zenith_angle',
       'L3_CH4_sensor_zenith_angle', 'year', 'month', 'week', 'day',
       'total_days_month', 'total_days']

In [ ]:
params = {
    'objective' :'regression',
    'learning_rate' : 0.02,
    'num_iterations': 30000,
    'max_bins': 50, 
    'max_depth' :7 ,
    'num_leaves' : 70,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'rmse' ,
     'min_data_in_leaf':5,
    'reg_lambda' :100
}


train_set = lgb.Dataset(train[columns], train['target'], silent=False,categorical_feature=['year','month','day','week'])
valid_set = lgb.Dataset(train[columns], train['target'], silent=False,categorical_feature=['year','month','day','week'])
model = lgb.train(params, train_set = train_set, num_boost_round=10000,early_stopping_rounds=500,verbose_eval=500, valid_sets=valid_set)

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds
[500]	valid_0's rmse: 22.5519
[1000]	valid_0's rmse: 18.5937
[1500]	valid_0's rmse: 16.0523
[2000]	valid_0's rmse: 14.0979
[2500]	valid_0's rmse: 12.4692
[3000]	valid_0's rmse: 11.0486
[3500]	valid_0's rmse: 9.80385
[4000]	valid_0's rmse: 8.71124
[4500]	valid_0's rmse: 7.76057
[5000]	valid_0's rmse: 6.8907
[5500]	valid_0's rmse: 6.13261
[6000]	valid_0's rmse: 5.45612
[6500]	valid_0's rmse: 4.86139
[7000]	valid_0's rmse: 4.34233
[7500]	valid_0's rmse: 3.88277
[8000]	valid_0's rmse: 3.47822
[8500]	valid_0's rmse: 3.11739
[9000]	valid_0's rmse: 2.79744
[9500]	valid_0's rmse: 2.51236
[10000]	valid_0's rmse: 2.26078
[10500]	valid_0's rmse: 2.04026
[11000]	valid_0's rmse: 1.8391
[11500]	valid_0's rmse: 1.66076
[12000]	valid_0's rmse: 1.5002
[12500]	valid_0's rmse: 1.35969
[13000]	valid_0's rmse: 1.2325
[13500]	valid_0's rmse: 1.11704
[14000]	valid_0's rmse: 1.01418
[14500]	valid_0's rmse: 0.920496
[15000]	valid_0's rmse: 0.836494


In [ ]:
prediction = model.predict(test[columns])
prediction

### Plotting feature importance

In [ ]:
lgb.plot_importance(model, figsize=(20,40))
plt.show()

### Create Submission

In [ ]:
#Submission
submit = pd.DataFrame({'Place_ID X Date': test['Place_ID X Date'], 'target': prediction})
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv" , index=False)
    return FileLink(submission_name+".csv")
create_submission(submit, 'submit')